In [1]:
import pandas as pd
from langdetect import detect
import numpy as np
import matplotlib.pyplot as plt
import numpy

In [37]:
train = pd.read_csv("<location of repo>/aml-mlops-workshop/outputs/raw_data/raw_subset_train.csv")
test = pd.read_csv("<location of repo>/aml-mlops-workshop/outputs/raw_data/raw_subset_test.csv")

In [38]:
print(train)

Unnamed: 0                                               text  target
0           1969  From: IMAGING.CLUB@OFFICE.WANG.COM ("Imaging C...       1
1            416  From: mike@starburst.umd.edu (Michael F. Santa...       2
2           1707  From: higgins@fnalf.fnal.gov (Bill Higgins-- B...       2
3           1285  From: rosst@pogo.wv.tek.com (Ross Taylor) Subj...       3
4            804  From: nsmca@aurora.alaska.edu Subject: Re: Jem...       2
...          ...                                                ...     ...
1012        1920  From: dewey@risc.sps.mot.com (Dewey Henize) Su...       0
1013        1271  From: C.O.EGALON@LARC.NASA.GOV (CLAUDIO OLIVEI...       2
1014        1786  From: havardn@edb.tih.no (Haavard Nesse,o92a) ...       1
1015         632  From: pharvey@quack.kfu.com (Paul Harvey) Subj...       3
1016        1709  From: eapu207@orion.oac.uci.edu (John Peter Ko...       1

[1017 rows x 3 columns]


In [39]:
print(test)

Unnamed: 0                                               text  target
0           464  From: bil@okcforum.osrhe.edu (Bill Conner) Sub...       0
1           612  From: marshall@csugrad.cs.vt.edu (Kevin Marsha...       0
2          1077  From: acooper@mac.cc.macalstr.edu (Turin Turam...       0
3           820  From: livesey@solntze.wpd.sgi.com (Jon Livesey...       0
4           920  From: danb@shell.portal.com (Dan E Babcock) Su...       0
..          ...                                                ...     ...
671         775  From: cs89ssg@brunel.ac.uk (Sunil Gupta) Subje...       1
672         625  From: mathew <mathew@mantis.co.uk> Subject: Re...       0
673         156  From: jgreen@trumpet.calpoly.edu (James Thomas...       0
674         596  From: gorgen@ann-arbor.applicon.slb.com (David...       1
675         868  From: I3150101@dbstu1.rz.tu-bs.de (Benedikt Ro...       0

[676 rows x 3 columns]


In [6]:
# check the column types
print(train.text.dtype)
print(test.text.dtype)
print(train.target.dtype)
print(test.target.dtype)

object
object
int64
int64


In [5]:
# check if there are non/na values present in the data
print(train.isnull().all())
print(test.isnull().all())

Unnamed: 0    False
text          False
target        False
dtype: bool
Unnamed: 0    False
text          False
target        False
dtype: bool


In [6]:
# calculate for every sentence in the dataset the language
# note that this could take a couple of minutes
lang_train =  np.array([])
lang_test = np.array([])
for i in range(0, len(train)):
    lang_train =  np.append(lang_train,detect(train.text.values[i]))

for i in range(0, len(test)):
    lang_test = np.append(lang_test, detect(test.text.values[i]))

In [7]:
# check if all sentences are indeed english
test2 = lang_train == 'en'
test1 = lang_test == 'en'
print(test2.all())
print(test1.all())

True
True


In [13]:
# investigare the mean of classes and their standard deviation
# this is very usefull to see if we have balanced classes or 
classes = train.target.value_counts(normalize=True)
train_mean = classes.mean()
print(train_mean)
train_std = classes.std()
print(train_std)
if train_std > 0.05:
    print("WARNING: CLasses might be imbalanced")

0.25
0.04287821802571039


In [14]:
# do the same for the test data set
classes = test.target.value_counts(normalize=True)
test_mean = classes.mean()
print(test_mean)
test_std = classes.std()
print(test_std)
if train_std > 0.05:
    print("WARNING: CLasses might be imbalanced")


0.25
0.04204932072056773


In [40]:
# clean the data for further investigation
import string

# make every thing lower case
train.text = train.text.apply(lambda x: x.lower())
test.text = test.text.apply(lambda x: x.lower())

# remove punctuation
translator = str.maketrans('', '', string.punctuation)
train.text = train.text.apply(
    lambda x: x.translate(translator))
test.text = test.text.apply(
    lambda x: x.translate(translator))

# remoce digits
train.text = train.text.apply(
    lambda x: x.translate(string.digits))
test.text = test.text.apply(
    lambda x: x.translate(string.digits))


In [41]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

stop = stopwords.words('english')

clean_text_train = train['text'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))
clean_text_test = test['text'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\mideboer.EUROPE\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [13]:
# calculate the average word length
def avg_word(sentence):
  words = sentence.split()
  return (sum(len(word) for word in words)/len(words))

train['avg_word'] = train['text'].apply(lambda x: avg_word(x))
train[['text','avg_word']].head()

,text,avg_word
0,"From: IMAGING.CLUB@OFFICE.WANG.COM (""Imaging C...",5.563025
1,From: mike@starburst.umd.edu (Michael F. Santa...,5.411602
2,From: higgins@fnalf.fnal.gov (Bill Higgins-- B...,5.710183
3,From: rosst@pogo.wv.tek.com (Ross Taylor) Subj...,5.866667
4,From: nsmca@aurora.alaska.edu Subject: Re: Jem...,5.317568


In [14]:
# calcuate the average word length
test['avg_word'] = test['text'].apply(lambda x: avg_word(x))
test[['text','avg_word']].head()

,text,avg_word
0,From: bil@okcforum.osrhe.edu (Bill Conner) Sub...,5.047826
1,From: marshall@csugrad.cs.vt.edu (Kevin Marsha...,4.969086
2,From: acooper@mac.cc.macalstr.edu (Turin Turam...,5.482517
3,From: livesey@solntze.wpd.sgi.com (Jon Livesey...,5.173160
4,From: danb@shell.portal.com (Dan E Babcock) Su...,6.191011


In [15]:
# print the mean average word length
print(train['avg_word'].mean())
print(test['avg_word'].mean())

5.566470149229751
5.52161248317771


In [16]:
# calculate the number of stop words
train['stopwords'] = train['text'].apply(lambda x: len([x for x in x.split() if x in stop]))
train[['text','stopwords']].head()

,text,stopwords
0,"From: IMAGING.CLUB@OFFICE.WANG.COM (""Imaging C...",36
1,From: mike@starburst.umd.edu (Michael F. Santa...,75
2,From: higgins@fnalf.fnal.gov (Bill Higgins-- B...,97
3,From: rosst@pogo.wv.tek.com (Ross Taylor) Subj...,23
4,From: nsmca@aurora.alaska.edu Subject: Re: Jem...,44


In [17]:
# calculate the number of stop words
test['stopwords'] = test['text'].apply(lambda x: len([x for x in x.split() if x in stop]))
test[['text','stopwords']].head()

,text,stopwords
0,From: bil@okcforum.osrhe.edu (Bill Conner) Sub...,86
1,From: marshall@csugrad.cs.vt.edu (Kevin Marsha...,276
2,From: acooper@mac.cc.macalstr.edu (Turin Turam...,198
3,From: livesey@solntze.wpd.sgi.com (Jon Livesey...,77
4,From: danb@shell.portal.com (Dan E Babcock) Su...,20


In [18]:
# calculate the average number of stop words
print(train['stopwords'].mean())
print(test['stopwords'].mean())

96.90855457227138
111.84319526627219


In [22]:
# calculate the sentiment of a text message
from textblob import TextBlob
train['sentiment'] = train['text'].apply(lambda x: TextBlob(x).sentiment[0] )
train[['text','sentiment']].head()

,text,sentiment
0,"From: IMAGING.CLUB@OFFICE.WANG.COM (""Imaging C...",0.097803
1,From: mike@starburst.umd.edu (Michael F. Santa...,0.073117
2,From: higgins@fnalf.fnal.gov (Bill Higgins-- B...,0.158598
3,From: rosst@pogo.wv.tek.com (Ross Taylor) Subj...,-0.113333
4,From: nsmca@aurora.alaska.edu Subject: Re: Jem...,0.080385


In [23]:
# calculate the sentiment of a text message
from textblob import TextBlob
test['sentiment'] = test['text'].apply(lambda x: TextBlob(x).sentiment[0] )
test[['text','sentiment']].head()

,text,sentiment
0,From: bil@okcforum.osrhe.edu (Bill Conner) Sub...,0.031373
1,From: marshall@csugrad.cs.vt.edu (Kevin Marsha...,-0.111508
2,From: acooper@mac.cc.macalstr.edu (Turin Turam...,0.075206
3,From: livesey@solntze.wpd.sgi.com (Jon Livesey...,-0.297672
4,From: danb@shell.portal.com (Dan E Babcock) Su...,0.139881


In [24]:
# print the average sentiment of the data set
print(train['sentiment'].mean())
print(test['sentiment'].mean())

0.06749869848993294
0.08094657902434026


In [42]:
# calcutate tfidf for every catergory in the data
from collections import Counter
df2= pd.DataFrame()
df1 = pd.DataFrame()

for classes in range(0,4):
    vocab = Counter()
    df_train = clean_text_train[train.target == classes]
    for text in df_train:
        for word in text.split(' '):
            vocab[word.lower()] += 1

    word1 = 'word' + str(classes)
    count = 'count' + str(classes)
    idf = 'idf' + str(classes)
    tfidf = 'tfidf' + str(classes)

    df = pd.DataFrame(list(vocab.items()), columns=[word1, count])

    for i,word in enumerate(df[word1]):
        df.loc[i, idf ] = np.log(df_train.shape[0]/(len(df_train[df_train.str.contains(word)]))) 

    array = df[count] * df[idf]
    df[tfidf] = array

    df1 = df.sort_values(tfidf, ascending=False, ignore_index=True)
    df2[word1] = df1[word1]
    df2[count] = df1[count]
    df2[idf]  =df1[idf]
    df2[tfidf] = df1[tfidf]

In [65]:
# print tfidf for every caterogy in the data.
df2.head(20)

,word0,count0,idf0,tfidf0,word1,count1,idf1,tfidf1,word2,count2,idf2,tfidf2,word3,count3,idf3,tfidf3
0,god,384,0.950482,364.985125,jpeg,220,3.278859,721.348876,launch,206,1.599039,329.402076,judas,75,3.912023,293.401725
1,jesus,146,2.104975,307.326327,image,352,1.599216,562.924158,space,554,0.500427,277.236512,jesus,194,1.406497,272.860431
2,people,294,0.918394,270.007772,images,160,2.275556,364.089027,data,129,1.995921,257.473753,god,207,0.941609,194.912968
3,system,132,1.916923,253.033785,files,156,1.939084,302.497133,would,265,0.863692,228.878293,people,217,0.879477,190.846457
4,matthew,67,3.526361,236.266155,gif,106,2.344549,248.522225,satellite,94,2.327278,218.764104,objective,76,2.353878,178.894757
5,atheism,135,1.711071,230.994525,software,137,1.784934,244.535890,commercial,71,2.769110,196.606843,would,236,0.744440,175.687952
6,bible,121,1.888752,228.538960,data,110,2.180246,239.827086,shuttle,100,1.788281,178.828120,tyre,51,3.352407,170.972768
7,argument,129,1.758699,226.872120,file,171,1.400088,239.414994,program,102,1.689190,172.297411,greek,54,3.101093,167.459011
8,atheists,204,1.090244,222.409784,also,140,1.706462,238.904664,moon,107,1.565138,167.469729,bible,105,1.584745,166.398256
9,would,256,0.808832,207.060884,version,135,1.725510,232.943861,dont,118,1.396802,164.822676,morality,90,1.832581,164.932332
